## Imports

In [9]:
from pprint import pprint
import json
import pandas as pd
import re

## Laden van data
We hebben een mini file gemaakt, is sneller met testen aangezien alle 600.000 tweets nogal wat ram vereist

Dit is dus de data ZONDER de bots

Link mini file:
https://drive.google.com/file/d/1391zXxC0msq8oAFlIKjKKlvkDx0PZdKz/view?usp=sharing

Link grote:
https://drive.google.com/file/d/1EM7Z0I8BVwd_ogAUTKaHcbxXRLnQYcTh/view?usp=sharing


In [10]:
%%time
# Verander in de regel hieronder de locatie van de data
tweets = pd.read_json("mini_twitter_geotagged_clean.json", orient='records', lines=True)

Wall time: 17.5 s


In [11]:
len(tweets)

46811

## Vinden van mentions en hashtags
Mentions en hashtags zijn in een aparte kolom geplaatst, zodat deze makkelijk te gebruiken zijn en te koppelen aan Trump en Hilary

In [28]:
# Create a new coloumn that contains a list with all the mentions used in the tweet
def mentionfinder(row):
    mentions = re.findall('(?<![@\w])@(\w{1,25})', row[27])
    mentions = tuple(mentions)
    return mentions

tweets['Mentions'] = tweets.apply(lambda row: mentionfinder(row), axis=1)

In [29]:
# Create a new coloumn that contains a list with all the hastags used in the tweet
def hashtagfinder(row):
    hashtags = re.findall('(?:(?<=\s)|(?<=^))#.*?(?=\s|$)', row[27]) 
    hashtags = tuple(hashtags)
    return hashtags

tweets['Hashtags'] = tweets.apply(lambda row: hashtagfinder(row), axis=1)

## Data
Kolommen die wij hebben toegevoegt: 
* CleanText, bevat de tekst zonder hyperlinks, mentions en hashtags
* Mentions, bevat mentions
* Hashtags, bevat hashtags  

Opvallend:
* Veel hebben geen coordinaten, contributers etc. Er mist vrij veel data ookal zeiden ze dat deze data gegeotagd was..

In [30]:
tweets.head()

,CleanText,contributors,coordinates,created_at,entities,extended_entities,favorite_count,favorited,filter_level,geo,...,quoted_status_id_str,retweet_count,retweeted,source,text,timestamp_ms,truncated,user,Mentions,Hashtags
0,\n\n\nALL IN COLLUSION TOGETHER \n\n\n\n\n\n\n...,NaN,None,2016-08-12 10:04:02,"{'hashtags': [{'text': 'NOJUSTICE', 'indices':...",None,0,False,low,None,...,NaN,0,False,"<a href=""http://twitter.com/download/android"" ...",@BarackObama \n@FBI\n@LORETTALYNCH \nALL IN CO...,2016-08-12 10:04:02.194,False,"{'id': 82496193, 'id_str': '82496193', 'name':...","(BarackObama, FBI, LORETTALYNCH, realDonaldTrump)","(#NOJUSTICE, #TrumpPence)"
1,he will do in one year all the things you shou...,NaN,None,2016-08-12 10:04:21,"{'hashtags': [], 'urls': [], 'user_mentions': ...",None,0,False,low,None,...,NaN,0,False,"<a href=""http://twitter.com/download/iphone"" r...",@HillaryClinton he will do in one year all the...,2016-08-12 10:04:21.125,False,"{'id': 44032624, 'id_str': '44032624', 'name':...","(HillaryClinton,)",()
2,clear deliberately throwing this racein 2007 h...,NaN,None,2016-08-12 10:04:30,"{'hashtags': [{'text': 'CNN', 'indices': [0, 4...",None,0,False,low,None,...,NaN,0,False,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",#CNN #newday clear #Trump deliberately throwin...,2016-08-12 10:04:30.035,False,"{'id': 769208504, 'id_str': '769208504', 'name...",(),"(#CNN, #newday, #Trump, #ISIS)"
3,you wouldnt recognize a lie if it came from y...,NaN,None,2016-08-12 10:04:46,"{'hashtags': [{'text': 'NeverTrump', 'indices'...",None,0,False,low,None,...,7.639507e+17,0,False,"<a href=""http://twitter.com/download/iphone"" r...","@realDonaldTrump, you wouldn't recognize a lie...",2016-08-12 10:04:46.265,False,"{'id': 41043316, 'id_str': '41043316', 'name':...","(realDonaldTrump,)","(#NeverTrump,)"
4,Kid you know suing someone Thats the most beau...,NaN,None,2016-08-12 10:04:48,"{'hashtags': [], 'urls': [], 'user_mentions': ...",None,0,False,low,None,...,NaN,0,False,"<a href=""http://twitter.com/download/iphone"" r...","""Kid, you know, suing someone? Thats the most ...",2016-08-12 10:04:48.229,False,"{'id': 762090248159371264, 'id_str': '76209024...","(funnyordie, realDonaldTrump)",()


In [18]:
tweets.favorited.value_counts()

False    46811
Name: favorited, dtype: int64

In [21]:
tweets.retweeted.value_counts()

False    46811
Name: retweeted, dtype: int64

## Combinaties van hoe mentions en hashtags gebruikt worden

In [31]:
tweets.Mentions.value_counts()

()                                                                                 10969
(realDonaldTrump,)                                                                 10123
(HillaryClinton,)                                                                   2235
(realDonaldTrump, CNN)                                                               508
(realDonaldTrump, nytimes)                                                           461
(HillaryClinton, realDonaldTrump)                                                    329
(realDonaldTrump, HillaryClinton)                                                    217
(realDonaldTrump, seanhannity, GovMikeHuckabee)                                      175
(FoxNews, HillaryClinton)                                                            120
(FoxNews, realDonaldTrump)                                                           111
(politico, realDonaldTrump)                                                          110
(realDonaldTrump, zul

In [33]:
tweets.Hashtags.value_counts()

()                                                                                                                                                            29207
(#Trump,)                                                                                                                                                      1400
(#NeverTrump,)                                                                                                                                                  633
(#MAGA,)                                                                                                                                                        434
(#ImWithHer,)                                                                                                                                                   366
(#TrumpPence16,)                                                                                                                                                315
(#NeverHillary,)